# WW2 Fleet Battle — mini “World of Warships” web game
This notebook generates a single-page web game (`index.html`) in the same folder and opens it in Google Chrome.

**Controls**
- `1` `2` `3` `4`: select ship (3 destroyers + 1 battleship)
- `W/S`: throttle forward/back (high inertia; ships accelerate slowly)
- `A/D`: turn left/right
- Aim with your mouse (turret follows cursor)
- Left click: fire shells at the clicked point (impact-only)
- `Q`: fire torpedo (if off cooldown)
- `E`: smoke screen (countermeasure)
- `F`: fire extinguisher (stop onboard fire)
- `R`: restart level

**Goal**
Sink the enemy fleet to advance levels (AI gets tougher).

In [90]:
from __future__ import annotations
from pathlib import Path
import os
import re

BASE = Path(r"C:\Users\fabla\OneDrive - Universidad Politécnica de Cartagena\freetime\AFK")
out_path = BASE / "index.html"

def replace_once(text: str, pattern: str, repl, *, flags: int = 0, label: str = "") -> str:
    new_text, n = re.subn(pattern, repl, text, count=1, flags=flags)
    if n != 1:
        raise RuntimeError(f"Expected 1 match for {label or pattern!r}, got {n}")
    return new_text

def replace_maybe(text: str, pattern: str, repl, *, flags: int = 0) -> tuple[str, int]:
    return re.subn(pattern, repl, text, flags=flags)

html = out_path.read_text(encoding="utf-8")

# 1) CV can also launch recon (Shift+T), BB stays same
if "if (e.shiftKey) tryRecon(s);" not in html:
    html = replace_once(
        html,
        r"if \(e\.code === 'KeyT'\) \{\s*const s = state\.player\[state\.selected\];\s*if \(!s\) return;\s*if \(s\.kind\.type === 'BB'\) tryRecon\(s\);\s*else if \(s\.kind\.type === 'CV'\) tryLaunchFighters\(s\);\s*\}",
        "if (e.code === 'KeyT') {\n"
        "      const s = state.player[state.selected];\n"
        "      if (!s) return;\n"
        "      if (s.kind.type === 'BB') tryRecon(s);\n"
        "      else if (s.kind.type === 'CV') {\n"
        "        if (e.shiftKey) tryRecon(s);\n"
        "        else tryLaunchFighters(s);\n"
        "      }\n"
        "    }",
        flags=re.MULTILINE,
        label="KeyT handler"
    )

# 2) Allow recon from CV too
if "Battleships/Carriers" not in html:
    html = replace_once(
        html,
        r"if \(ship\.kind\.type !== 'BB'\) \{ if \(!silent && ship\.team === 'P'\) showMsg\('Recon plane only available on Battleships'\); return; \}",
        "if (ship.kind.type !== 'BB' && ship.kind.type !== 'CV') { if (!silent && ship.team === 'P') showMsg('Recon plane only available on Battleships/Carriers'); return; }",
        label="tryRecon ship kind gate"
    )

# 3) Islands controlled by US: only spawn US (P) coastal batteries
html = replace_maybe(
    html,
    r"placeTeam\('P'\);\s*placeTeam\('E'\);",
    "placeTeam('P');",
)[0]

if "const maxPerTeam = 7;" not in html:
    html = replace_once(
        html,
        r"const maxPerTeam = 6;",
        "// Islands are controlled by the US: only spawn US (P) batteries\n    const maxPerTeam = 7;",
        label="spawnCoastalBatteries maxPerTeam"
    )
html = replace_maybe(
    html,
    r"tag: 'Coastal Battery'",
    "tag: 'US Coastal Battery'",
 )[0]

# 4) Bots avoid hostile battery zones (enemy avoids US batteries)
INSERT_AVOID = (
    "function avoidHostileBatteries(ship, tx, ty) {\n"
    "    if (!ship || !ship.alive) return { x: tx, y: ty };\n"
    "    const hostile = state.batteries.filter(b => b.alive && b.team !== ship.team);\n"
    "    if (!hostile.length) return { x: tx, y: ty };\n"
    "    let x = tx, y = ty;\n"
    "    const danger = BATTERY.gunRange * 0.92;\n"
    "    for (const b of hostile) {\n"
    "      const dx = x - b.x, dy = y - b.y;\n"
    "      const d = Math.hypot(dx, dy);\n"
    "      if (d < danger) {\n"
    "        const a = (d > 1e-3) ? Math.atan2(dy, dx) : rand(0, TAU);\n"
    "        x = b.x + Math.cos(a) * danger;\n"
    "        y = b.y + Math.sin(a) * danger;\n"
    "      }\n"
    "    }\n"
    "    return { x: clamp(x, 0, WORLD.w), y: clamp(y, 0, WORLD.h) };\n"
    "}\n\n"
 )
if "function avoidHostileBatteries(" not in html:
    html = replace_once(
        html,
        r"function pickTargetDetectable\(fromShip, candidates\) \{[\s\S]*?\}\n\n\s*function aiStep\(",
        lambda m: m.group(0).replace("function aiStep(", INSERT_AVOID + "function aiStep("),
        flags=re.MULTILINE,
        label="insert avoidHostileBatteries"
    )

# Formation follow target avoidance
html = replace_maybe(
    html,
    r"const fp = formationPoint\(leader, ship\);\s*const dForm = steerToPoint\(ship, clamp\(fp\.x, 0, WORLD\.w\), clamp\(fp\.y, 0, WORLD\.h\), dt, ship\.kind\.maxSpeed\);",
    "const fp = formationPoint(leader, ship);\n"
    "      let tgt = { x: clamp(fp.x, 0, WORLD.w), y: clamp(fp.y, 0, WORLD.h) };\n"
    "      tgt = avoidHostileBatteries(ship, tgt.x, tgt.y);\n"
    "      const dForm = steerToPoint(ship, tgt.x, tgt.y, dt, ship.kind.maxSpeed);",
 )[0]

# Patrol avoidance: adjust waypoint before steering
if "avoidHostileBatteries(ship, ship.ai.wpX, ship.ai.wpY)" not in html:
    html = replace_once(
        html,
        r"if \(!best\) \{[\s\S]*?ship\.ai\.wpUntil = state\.time \+ rand\(4\.0, 7\.5\);\s*\}\s*const desired = angleTo\(ship\.x, ship\.y, ship\.ai\.wpX, ship\.ai\.wpY\);",
        lambda m: m.group(0).replace(
            "const desired = angleTo(ship.x, ship.y, ship.ai.wpX, ship.ai.wpY);",
            "const adj = avoidHostileBatteries(ship, ship.ai.wpX, ship.ai.wpY);\n"
            "      ship.ai.wpX = adj.x; ship.ai.wpY = adj.y;\n"
            "      const desired = angleTo(ship.x, ship.y, ship.ai.wpX, ship.ai.wpY);",
        ),
        flags=re.MULTILINE,
        label="aiStep patrol avoidance"
    )

# Orbit avoidance: adjust tx/ty before final desired angle
if "const adj = avoidHostileBatteries(ship, tx, ty);" not in html:
    html = replace_once(
        html,
        r"let tx = best\.x, ty = best\.y;[\s\S]*?\}\s*const desired = angleTo\(ship\.x, ship\.y, tx, ty\);",
        lambda m: m.group(0).replace(
            "const desired = angleTo(ship.x, ship.y, tx, ty);",
            "const adj = avoidHostileBatteries(ship, tx, ty);\n"
            "    tx = adj.x; ty = adj.y;\n"
            "    const desired = angleTo(ship.x, ship.y, tx, ty);",
        ),
        flags=re.MULTILINE,
        label="aiStep orbit avoidance"
    )

# 5) Minimap: make grey zone obvious (land + battery danger rings)
if "// Grey zone: land + coastal battery danger areas" not in html:
    html = replace_once(
        html,
        r"const toR = \(x, y\) => \(\{ x: rx \+ \(x / WORLD\.w\) \* rw, y: ry \+ \(y / WORLD\.h\) \* rh \}\);",
        "const toR = (x, y) => ({ x: rx + (x / WORLD.w) * rw, y: ry + (y / WORLD.h) * rh });\n"
        "    // Grey zone: land + coastal battery danger areas\n"
        "    const m = mapById(state.mapId);\n"
        "    // Land silhouettes\n"
        "    ctx.save();\n"
        "    ctx.globalAlpha = 0.36;\n"
        "    ctx.fillStyle = 'rgba(170,170,170,0.45)';\n"
        "    for (const c of m.land) {\n"
        "      const q = toR(c.x, c.y);\n"
        "      const rs = ((rw + rh) * 0.5) / ((WORLD.w + WORLD.h) * 0.5);\n"
        "      const r = Math.max(1.2, c.r * rs);\n"
        "      ctx.beginPath(); ctx.arc(q.x, q.y, r, 0, TAU); ctx.fill();\n"
        "    }\n"
        "    ctx.restore();\n"
        "    // Battery danger circles (US-controlled islands)\n"
        "    ctx.save();\n"
        "    ctx.globalAlpha = 0.22;\n"
        "    ctx.fillStyle = 'rgba(190,190,190,0.22)';\n"
        "    ctx.strokeStyle = 'rgba(220,220,220,0.28)';\n"
        "    ctx.lineWidth = 1;\n"
        "    for (const b of state.batteries) {\n"
        "      if (!b.alive) continue;\n"
        "      if (b.team !== 'P') continue;\n"
        "      const q = toR(b.x, b.y);\n"
        "      const rs = ((rw + rh) * 0.5) / ((WORLD.w + WORLD.h) * 0.5);\n"
        "      const r = Math.max(1.2, b.kind.gunRange * rs);\n"
        "      ctx.beginPath(); ctx.arc(q.x, q.y, r, 0, TAU); ctx.fill();\n"
        "      ctx.beginPath(); ctx.arc(q.x, q.y, r, 0, TAU); ctx.stroke();\n"
        "    }\n"
        "    ctx.restore();",
        flags=re.MULTILINE,
        label="drawRadar overlay"
    )

# 6) DDs escort capitals ahead + predictive screen position
if "// Predictive escort positioning" not in html:
    NEW_SET_FORMATION = (
        "function setFormation(teamArr, formKey) {\n"
        "    const leader = pickCapitalLeader(teamArr) || teamArr[0];\n"
        "    if (!leader) return;\n"
        "    const pri = (s) => (s.kind.type === 'CV' ? 0 : (s.kind.type === 'BB' ? 1 : (s.kind.type === 'DD' ? 2 : (s.kind.type === 'TB' ? 3 : 4))));\n"
        "    const others = teamArr.filter(s => s !== leader).slice().sort((a,b) => pri(a) - pri(b));\n"
        "    const ordered = [leader].concat(others);\n"
        "    const offs = formationOffsets(formKey, ordered.length);\n"
        "    const sep = 92 * MAP_SCALE;\n"
        "    const side = 70 * MAP_SCALE;\n"
        "    const bb = (leader.kind.type === 'CV') ? ordered.find(s => s.kind.type === 'BB') : null;\n"
        "    // DD screen: put destroyers AHEAD of the capital, forming a forward picket line\n"
        "    let ddFrontIdx = 0;\n"
        "    for (let i=0;i<ordered.length;i++) {\n"
        "      const s = ordered[i];\n"
        "      s.formKey = formKey;\n"
        "      s.formRank = i;\n"
        "      let f = offs[Math.min(i, offs.length-1)][0];\n"
        "      let r = offs[Math.min(i, offs.length-1)][1];\n"
        "      if (bb && s === bb) { f = -sep; r = 0; }\n"
        "      if ((leader.kind.type === 'CV' || leader.kind.type === 'BB') && s !== leader && s.kind.type === 'DD') {\n"
        "        const row = Math.floor(ddFrontIdx / 2) + 1;\n"
        "        const lr = (ddFrontIdx % 2 === 0) ? -1 : 1;\n"
        "        f = row * sep * 0.95;\n"
        "        r = lr * side * (0.95 + 0.25 * (row - 1));\n"
        "        ddFrontIdx++;\n"
        "      }\n"
        "      s.formF = f;\n"
        "      s.formR = r;\n"
        "    }\n"
        "  }\n"
    )
    html = replace_once(
        html,
        r"function setFormation\(teamArr, formKey\) \{[\s\S]*?\}\n\s*(function formationPoint\(leader, ship\) \{)",
        lambda m: NEW_SET_FORMATION + "\n\n  " + m.group(1),
        flags=re.MULTILINE,
        label="replace setFormation"
    )

    NEW_FORMATION_POINT = (
        "function formationPoint(leader, ship) {\n"
        "    // Predictive escort positioning: aim for where the leader WILL BE, not where it is now\n"
        "    const sp = Math.hypot(leader.vx || 0, leader.vy || 0);\n"
        "    const frac = clamp(sp / Math.max(1, leader.kind.maxSpeed || 1), 0, 1);\n"
        "    const leadT = 0.90 + 1.20 * frac;\n"
        "    const baseX = leader.x + (leader.vx || 0) * leadT;\n"
        "    const baseY = leader.y + (leader.vy || 0) * leadT;\n"
        "    const h = (sp > 6) ? Math.atan2(leader.vy || 0, leader.vx || 0) : leader.heading;\n"
        "    const ca = Math.cos(h), sa = Math.sin(h);\n"
        "    const nx = -sa, ny = ca;\n"
        "    const f = ship.formF || 0, r = ship.formR || 0;\n"
        "    return { x: baseX + ca*f + nx*r, y: baseY + sa*f + ny*r };\n"
        "  }\n"
    )
    html = replace_once(
        html,
        r"function formationPoint\(leader, ship\) \{[\s\S]*?\}\n\s*(function steerToPoint\(ship, tx, ty, dt, speedMax\) \{)",
        lambda m: NEW_FORMATION_POINT + "\n\n  " + m.group(1),
        flags=re.MULTILINE,
        label="replace formationPoint"
    )

# 7) Ships avoid shore: land-aware steering + less 'stuck' grounding
if "function avoidLandTarget(" not in html:
    AVOID_LAND = (
        "function avoidLandTarget(ship, tx, ty) {\n"
        "    // Keep AI from steering into islands/shore (simple circle avoidance).\n"
        "    let x = tx, y = ty;\n"
        "    const m = mapById(state.mapId);\n"
        "    const pad = (ship && ship.kind) ? (ship.kind.radius + 22) : 40;\n"
        "    // If target is inside land, push it outside.\n"
        "    for (let k = 0; k < 3; k++) {\n"
        "      const hit = inLand(x, y, pad);\n"
        "      if (!hit) break;\n"
        "      const dx = x - hit.x, dy = y - hit.y;\n"
        "      const d = Math.hypot(dx, dy) || 1;\n"
        "      const want = hit.r + pad;\n"
        "      x = hit.x + (dx / d) * want;\n"
        "      y = hit.y + (dy / d) * want;\n"
        "    }\n"
        "    // If the segment to the target passes too close to land, sidestep around it.\n"
        "    const sx = ship.x, sy = ship.y;\n"
        "    const vx = x - sx, vy = y - sy;\n"
        "    const vv = vx*vx + vy*vy;\n"
        "    if (vv > 1e-6) {\n"
        "      for (const c of m.land) {\n"
        "        const margin = c.r + pad;\n"
        "        const t = clamp(((c.x - sx)*vx + (c.y - sy)*vy) / vv, 0, 1);\n"
        "        const cx = sx + vx*t, cy = sy + vy*t;\n"
        "        const dx = cx - c.x, dy = cy - c.y;\n"
        "        const d = Math.hypot(dx, dy);\n"
        "        if (d < margin) {\n"
        "          // Choose a tangent direction that moves us away from the island.\n"
        "          const nx = (d > 1e-3) ? (dx / d) : 1;\n"
        "          const ny = (d > 1e-3) ? (dy / d) : 0;\n"
        "          const tx1 = -ny, ty1 = nx;\n"
        "          const tx2 = ny, ty2 = -nx;\n"
        "          const sgn = ((vx*tx1 + vy*ty1) > (vx*tx2 + vy*ty2)) ? 1 : -1;\n"
        "          const off = (margin - d) + 35;\n"
        "          x += (sgn > 0 ? tx1 : tx2) * off;\n"
        "          y += (sgn > 0 ? ty1 : ty2) * off;\n"
        "        }\n"
        "      }\n"
        "    }\n"
        "    return { x: clamp(x, 0, WORLD.w), y: clamp(y, 0, WORLD.h) };\n"
        "  }\n\n"
    )
    html = replace_once(
        html,
        r"function steerToPoint\(ship, tx, ty, dt, speedMax\) \{",
        AVOID_LAND + "function steerToPoint(ship, tx, ty, dt, speedMax) {",
        label="insert avoidLandTarget"
    )

if "avoidLandTarget(ship, tx, ty)" not in html:
    html = replace_once(
        html,
        r"function steerToPoint\(ship, tx, ty, dt, speedMax\) \{\s*const desired = angleTo\(ship\.x, ship\.y, tx, ty\);",
        "function steerToPoint(ship, tx, ty, dt, speedMax) {\n"
        "    const adj = avoidLandTarget(ship, tx, ty);\n"
        "    tx = adj.x; ty = adj.y;\n"
        "    const desired = angleTo(ship.x, ship.y, tx, ty);",
        flags=re.MULTILINE,
        label="steerToPoint land-aware"
    )

# Make grounding less sticky: bounce/dampen instead of hard stop
if "ship.vx = 0; ship.vy = 0;" in html and "vn < 0" not in html:
    html = replace_once(
        html,
        r"ship\.vx = 0; ship\.vy = 0;",
        "const nx = dx / d, ny = dy / d;\n"
        "    const vn = ship.vx * nx + ship.vy * ny;\n"
        "    if (vn < 0) {\n"
        "      ship.vx -= vn * nx * 1.8;\n"
        "      ship.vy -= vn * ny * 1.8;\n"
        "      ship.vx *= 0.35;\n"
        "      ship.vy *= 0.35;\n"
        "    }",
        label="resolveShipLand bounce"
    )

# 8) Recon planes are per-ship (no team/global singleton)
if "recons:" not in html:
    # state init: replace reconP/E fields with a recon array
    html = replace_once(
        html,
        r"reconP: null, reconE: null, reconCdP: 0, reconCdE: 0, ended: false,",
        "recons: [], ended: false,",
        label="state recon fields"
    )
    # resetWorld: clear recons
    html = replace_once(
        html,
        r"state\.reconP = null;\s*state\.reconE = null;\s*state\.reconCdP = 0;\s*state\.reconCdE = 0;",
        "state.recons.length = 0;",
        flags=re.MULTILINE,
        label="resetWorld recon reset"
    )

    NEW_TRY_RECON = (
        "function tryRecon(ship, silent = false) {\n"
        "    if (!ship || !ship.alive) return;\n"
        "    if (ship.kind.type !== 'BB' && ship.kind.type !== 'CV') {\n"
        "      if (!silent && ship.team === 'P') showMsg('Recon plane only available on Battleships/Carriers');\n"
        "      return;\n"
        "    }\n"
        "    if (!('reconCd' in ship)) ship.reconCd = 0;\n"
        "    if (!state.recons) state.recons = [];\n"
        "    for (const r of state.recons) {\n"
        "      if (r && r.homeShip === ship) {\n"
        "        if (!silent && ship.team === 'P') showMsg('Recon plane already airborne');\n"
        "        return;\n"
        "      }\n"
        "    }\n"
        "    if (ship.reconCd > 0) {\n"
        "      if (!silent && ship.team === 'P') showMsg(`Recon rearming ${ship.reconCd.toFixed(1)}s`);\n"
        "      return;\n"
        "    }\n"
        "    const wp = reconPickWaypoint();\n"
        "    const team = ship.team;\n"
        "    const r = {\n"
        "      team, x: ship.x, y: ship.y, a: ship.heading,\n"
        "      vx: Math.cos(ship.heading) * RECON.speed, vy: Math.sin(ship.heading) * RECON.speed,\n"
        "      homeShip: ship, homeX: ship.x, homeY: ship.y,\n"
        "      ttl: RECON.ttl, mode: 'patrol', wpX: wp.x, wpY: wp.y, wpUntil: state.time + rand(3.8, 6.2),\n"
        "      spotted: new Set(),\n"
        "    };\n"
        "    state.recons.push(r);\n"
        "    ship.reconCd = RECON.cooldown;\n"
        "    if (!silent && team === 'P') showMsg('Recon plane launched');\n"
        "  }\n"
    )
    html = replace_once(
        html,
        r"function tryRecon\(ship, silent = false\) \{[\s\S]*?\}\n\n\s*function reconStep\(",
        lambda m: NEW_TRY_RECON + "\n\n  function reconStep(" ,
        flags=re.MULTILINE,
        label="replace tryRecon"
    )

    NEW_RECON_STEP = (
        "function reconStep(dt) {\n"
        "    // Cooldowns are per ship (independent recon planes).\n"
        "    for (const s of state.player) { if (s && s.alive && s.reconCd > 0) s.reconCd = Math.max(0, s.reconCd - dt); }\n"
        "    for (const s of state.enemy) { if (s && s.alive && s.reconCd > 0) s.reconCd = Math.max(0, s.reconCd - dt); }\n"
        "    if (!state.recons || state.recons.length === 0) return;\n"
        "    const next = [];\n"
        "    for (const r of state.recons) {\n"
        "      if (!r) continue;\n"
        "      r.ttl -= dt;\n"
        "      if (r.ttl <= 0 && r.mode !== 'return') r.mode = 'return';\n"
        "      const targets = (r.team === 'P') ? state.enemy : state.player;\n"
        "      for (const e of targets) {\n"
        "        if (!e.alive) continue;\n"
        "        if (dist2(r.x, r.y, e.x, e.y) <= RECON_V2) { revealToTeam(e, r.team); r.spotted.add(e); }\n"
        "      }\n"
        "      const aliveTargets = targets.filter(e => e.alive);\n"
        "      if (r.mode === 'patrol') {\n"
        "        let allSpotted = aliveTargets.length > 0;\n"
        "        for (const e of aliveTargets) { if (!r.spotted.has(e)) { allSpotted = false; break; } }\n"
        "        if (allSpotted) r.mode = 'return';\n"
        "      }\n"
        "      const homeAlive = r.homeShip && r.homeShip.alive;\n"
        "      const homeX = homeAlive ? r.homeShip.x : r.homeX;\n"
        "      const homeY = homeAlive ? r.homeShip.y : r.homeY;\n"
        "      if (r.mode === 'patrol') {\n"
        "        if (state.time > r.wpUntil || dist2(r.x, r.y, r.wpX, r.wpY) < 140*140) {\n"
        "          const wp = reconPickWaypoint();\n"
        "          r.wpX = wp.x; r.wpY = wp.y;\n"
        "          r.wpUntil = state.time + rand(3.8, 6.2);\n"
        "        }\n"
        "        const desired = angleTo(r.x, r.y, r.wpX, r.wpY);\n"
        "        const turn = clamp(normAngle(desired - r.a), -1, 1);\n"
        "        r.a += turn * RECON.turnRate * dt;\n"
        "      } else {\n"
        "        const desired = angleTo(r.x, r.y, homeX, homeY);\n"
        "        const turn = clamp(normAngle(desired - r.a), -1, 1);\n"
        "        r.a += turn * RECON.turnRate * 1.05 * dt;\n"
        "      }\n"
        "      r.vx = Math.cos(r.a) * RECON.speed;\n"
        "      r.vy = Math.sin(r.a) * RECON.speed;\n"
        "      r.x += r.vx * dt;\n"
        "      r.y += r.vy * dt;\n"
        "      r.x = clamp(r.x, 0, WORLD.w);\n"
        "      r.y = clamp(r.y, 0, WORLD.h);\n"
        "      if (r.mode === 'return' && dist2(r.x, r.y, homeX, homeY) < 22*22) {\n"
        "        if (r.team === 'P') showMsg('Recon plane recovered');\n"
        "        continue;\n"
        "      }\n"
        "      next.push(r);\n"
        "    }\n"
        "    state.recons = next;\n"
        "  }\n"
    )
    html = replace_once(
        html,
        r"function reconStep\(dt\) \{[\s\S]*?\}\n\n\s*// ═══════════════════════════════════════════════════════════════════════════\n\s*// CARRIER FIGHTERS",
        NEW_RECON_STEP + "\n\n  // ═══════════════════════════════════════════════════════════════════════════\n  // CARRIER FIGHTERS",
        flags=re.MULTILINE,
        label="replace reconStep"
    )

    # drawRecon: render all state.recons
    html = replace_once(
        html,
        r"function drawRecon\(camX, camY\) \{[\s\S]*?\}\n\n\s*function drawAircraft\(camX, camY\) \{" ,
        "function drawRecon(camX, camY) {\n"
        "    const drawOne = (r) => {\n"
        "      if (!r) return;\n"
        "      const x = r.x - camX, y = r.y - camY;\n"
        "      if (x < -80 || y < -80 || x > VIEW.w + 80 || y > VIEW.h + 80) return;\n"
        "      ctx.save();\n"
        "      ctx.globalAlpha = 0.30;\n"
        "      ctx.fillStyle = (r.team === 'P') ? 'rgba(80,170,255,0.85)' : 'rgba(255,60,60,0.85)';\n"
        "      ctx.beginPath(); ctx.arc(x, y, 18, 0, TAU); ctx.fill();\n"
        "      ctx.globalAlpha = 0.85;\n"
        "      if (r.team === 'P') {\n"
        "        ctx.strokeStyle = 'rgba(80,170,255,0.85)';\n"
        "        ctx.lineWidth = 3.0;\n"
        "        ctx.beginPath(); ctx.arc(x, y, 18, 0, TAU); ctx.stroke();\n"
        "      } else {\n"
        "        ctx.strokeStyle = 'rgba(255,60,60,0.90)';\n"
        "        ctx.lineWidth = 3.2;\n"
        "        ctx.beginPath(); ctx.arc(x, y, 18, 0, TAU); ctx.stroke();\n"
        "        ctx.strokeStyle = 'rgba(255,255,255,0.90)';\n"
        "        ctx.lineWidth = 2.6;\n"
        "        ctx.beginPath(); ctx.arc(x, y, 14, 0, TAU); ctx.stroke();\n"
        "      }\n"
        "      ctx.globalAlpha = 0.95;\n"
        "      ctx.font = '22px system-ui, Segoe UI Emoji, Apple Color Emoji, Noto Color Emoji';\n"
        "      ctx.textAlign = 'center';\n"
        "      ctx.textBaseline = 'middle';\n"
        "      ctx.shadowColor = 'rgba(0,0,0,0.45)';\n"
        "      ctx.shadowBlur = 6;\n"
        "      ctx.fillText('🛩️', x, y);\n"
        "      ctx.restore();\n"
        "    };\n"
        "    if (!state.recons) return;\n"
        "    for (const r of state.recons) drawOne(r);\n"
        "  }\n"
        "\n"
        "  function drawAircraft(camX, camY) {",
        flags=re.MULTILINE,
        label="replace drawRecon"
    )

    # updateUI recon text: per selected ship
    if "const airborne = state.recons" not in html:
        html = replace_once(
            html,
            r"const rtxt = state\.reconP \? 'Recon: Airborne' : \(state\.reconCdP > 0 \? `Recon: \$\{state\.reconCdP\.toFixed\(1\)\}s`\s*: 'Recon: Ready \(T\)'\);",
            "const airborne = state.recons && state.recons.some(r => r && r.homeShip === selShip);\n"
            "      const cd = selShip.reconCd || 0;\n"
            "      const rtxt = airborne ? 'Recon: Airborne' : (cd > 0 ? `Recon: ${cd.toFixed(1)}s` : 'Recon: Ready (T)');",
            flags=re.MULTILINE,
            label="updateUI recon status"
        )

# 9) Trees: make island trees stable (no per-frame flicker)
if "// Tree/vegetation spots" in html and "deterministic" not in html:
    html = replace_once(
        html,
        r"// Tree/vegetation spots[\s\S]*?ctx\.restore\(\);\n\n\s*// Animated shoreline foam",
        "// Tree/vegetation spots (deterministic: avoids flicker)\n"
        "      ctx.globalAlpha = 0.32;\n"
        "      const treeCount = Math.floor(c.r / 18);\n"
        "      for (let i = 0; i < treeCount; i++) {\n"
        "        let u = Math.sin((c.x * 0.017) + (c.y * 0.023) + i * 1.77) * 43758.5453;\n"
        "        u = u - Math.floor(u);\n"
        "        let v = Math.sin((c.x * 0.031) - (c.y * 0.019) + i * 2.31) * 12345.678;\n"
        "        v = v - Math.floor(v);\n"
        "        const angle = u * TAU + c.x * 0.001;\n"
        "        const dist = c.r * (0.22 + v * 0.55);\n"
        "        const tx = c.x + Math.cos(angle) * dist;\n"
        "        const ty = c.y + Math.sin(angle) * dist;\n"
        "        const tr = 7 + (u * 6);\n"
        "        ctx.fillStyle = 'rgba(25,70,30,0.42)';\n"
        "        ctx.beginPath();\n"
        "        ctx.moveTo(tx, ty - tr);\n"
        "        ctx.lineTo(tx - tr*0.75, ty + tr*0.65);\n"
        "        ctx.lineTo(tx + tr*0.75, ty + tr*0.65);\n"
        "        ctx.closePath();\n"
        "        ctx.fill();\n"
        "      }\n"
        "      ctx.restore();\n\n"
        "      // Animated shoreline foam",
        flags=re.MULTILINE,
        label="stable trees"
    )

out_path.write_text(html, encoding="utf-8")
print(f"Wrote {out_path}")
print("- Per-ship recon planes (no shared limit)")
print("- Ships steer around shore; grounding is less sticky")
print("- Islands have stable tree graphics")
print("- Shift+T on CV launches recon; T launches fighters")
print("- US-only island batteries; bots avoid hostile battery zones")
print("- Minimap shows land + battery grey-zone rings")
print("- DDs screen ahead of capitals with predictive positioning")

# Convenience: open in browser
try:
    os.startfile(out_path)
except Exception as e:
    print("Could not auto-open:", e)

Wrote C:\Users\fabla\OneDrive - Universidad Politécnica de Cartagena\freetime\AFK\index.html
- Per-ship recon planes (no shared limit)
- Ships steer around shore; grounding is less sticky
- Islands have stable tree graphics
- Shift+T on CV launches recon; T launches fighters
- US-only island batteries; bots avoid hostile battery zones
- Minimap shows land + battery grey-zone rings
- DDs screen ahead of capitals with predictive positioning
